In [10]:
import numpy as np
import nibabel as nib
import pandas as pd
import time

from proteus.matrix import registration
from nilearn.masking import compute_epi_mask

# Load 4D images

In [6]:
subj_id='HC0040123'
# original subject from pierre exp
covar = pd.read_csv('/home/cdansereau/data/deepmotion/cobre/cobre_fmri_preprocess_nii_20161003/resample/fmri_'+subj_id+'_session1_run1_n_confounds.tsv.gz',sep='\t')
covar.columns
vol_nii = nib.load('/home/cdansereau/data/deepmotion/cobre/xp_2016_07_27_final/rest_'+subj_id+'.nii.gz')


vol = vol_nii.get_data()
Xmotion = covar.iloc[:,0:6].values
Xdrift  = covar.iloc[:,8:14].values
fd = covar['FD'].values

print vol.shape

(64, 64, 33, 150)


# Compute EPI mask

In [4]:
mask_b = compute_epi_mask(vol_nii).get_data().astype(bool)

# Normalize frames

In [8]:
scaling_factor = vol[mask_b].std()
vol_z = (vol - vol.mean(-1)[...,np.newaxis]) / scaling_factor

# Estimate motion parameters (affine transformation)

In [11]:
reload(registration)
start = time.time()
coreg_vol, transf, params = registration.fit(vol_nii.get_data()[:,:,:,:],np.copy(vol_nii.get_affine()),np.median(vol_nii.get_data(),3),np.copy(vol_nii.get_affine()),mask=mask_b,dowsamp_flag=False)
#coreg_vol, transf, params = registration.fit(vol_nii.get_data()[:,:,:,0],np.copy(vol_nii.get_affine()),np.median(vol_nii.get_data(),3),np.copy(vol_nii.get_affine()),mask=mask_b,dowsamp_flag=True)
#coreg_vol, transf, params = registration.fit(vol_nii.get_data()[:,:,:,0],vol_nii.get_affine(),gaussian_filter(np.median(vol_nii.get_data(),3),0.5,0),vol_nii.get_affine(),mask_b,dowsamp_flag=True)
print(time.time() - start)

843.78905797


In [16]:
np.array(params)#.shape

array([[ -6.87095242e-01,  -6.17670702e-01,  -4.12501627e-01,
         -4.86861993e-01,  -5.09677587e-01,  -4.87724073e-01,
         -5.09794807e-01,  -5.22239436e-01,  -4.28787619e-01,
         -4.28362842e-01,  -4.12109353e-01,  -3.86803337e-01,
         -3.67689980e-01,  -3.49155480e-01,  -3.34569077e-01,
         -3.41862014e-01,  -3.29504753e-01,  -3.27223627e-01,
         -3.15487858e-01,  -2.89522160e-01,  -2.75582784e-01,
         -2.69925113e-01,  -2.65427972e-01,  -2.41907602e-01,
         -2.35042663e-01,  -2.12468075e-01,  -2.08147464e-01,
         -1.78881728e-01,  -1.92655111e-01,  -1.84886144e-01,
         -2.11423858e-01,  -2.25901445e-01,  -2.30692098e-01,
         -2.38680863e-01,  -2.54495195e-01,  -2.53184145e-01,
         -2.41901540e-01,  -2.44153580e-01,  -2.39344977e-01,
         -2.35669260e-01,  -2.10976001e-01,  -2.06618924e-01,
         -1.84092133e-01,  -1.87459888e-01,  -1.65583442e-01,
         -1.43432805e-01,  -1.34309857e-01,  -1.25915789e-01,
        

# Export data to HDF5